In [ ]:
# default_exp tst_net_2

In [ ]:
#hide
# from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# export
import torch.nn as nn
import sys, torch
from functools import partial
from collections import OrderedDict

from model_constructor.layers import *

# Utils

In [ ]:
# export
act_fn = nn.ReLU(inplace=True)

def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)


# ResBlock

In [ ]:
# export
class ResBlock(nn.Module):
    def __init__(self, expansion, ni, nh, stride=1, sa=False,
                 conv_layer=ConvLayer, act_fn=act_fn, zero_bn=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sym=False):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, stride=stride)),
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False))
        ] if expansion == 1 else [
                   (f"conv_0",conv_layer(ni, nh, 1)),
                   (f"conv_1",conv_layer(nh, nh, 3, stride=stride)),
                   (f"conv_2",conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.pool = noop if stride==1 else pool
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.act_fn =act_fn

    def forward(self, x): return self.act_fn(self.convs(x) + self.idconv(self.pool(x)))

In [ ]:
# export
# class version
class _ResBlock(Constructor):
    def __init__(self, expansion=1, conv_layer=ConvLayer, act_fn=act_fn, zero_bn=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sym=False):
        super().__init__()
        self.__dict__.update(locals())
#         self.__dict__.update(self.__dict__.pop('kwargs'))
        
        
    def __call__(self,  ni, nh, stride=1, sa=False):
        return ResBlock(self.expansion, ni,nh,stride,sa, 
                self.conv_layer, self.act_fn, self.zero_bn, self.pool, self.sym)
    def __getattr__(self, k): 
        if hasattr(self, '_model'):
            return getattr(self._model, k)

In [ ]:
block = _ResBlock()

In [ ]:
block

_ResBlock

In [ ]:
block(64,64)

TypeError: 'bool' object is not callable

In [ ]:
ResBlock(1,64,64,sa=True)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
ResBlock(2,64,64,act_fn=nn.LeakyReLU())

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): LeakyReLU(negative_slope=0.01)
)

# NewResBlock

In [ ]:
# # export
# # Still no name - just New block YET!
# class NewResBlock(nn.Module):
#     def __init__(self, expansion, ni, nh, stride=1, 
#                  conv_layer=ConvLayer, act_fn=act_fn,
#                  pool=nn.AvgPool2d(2, ceil_mode=True), sa=False,sym=False, zero_bn=True):
#         super().__init__()
#         nf,ni = nh*expansion,ni*expansion
#         self.reduce = noop if stride==1 else pool
#         layers  = [(f"conv_0", conv_layer(ni, nh, 3, stride=1)), # stride 1 !!!
#                    (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False))
#         ] if expansion == 1 else [
#                    (f"conv_0",conv_layer(ni, nh, 1)),
#                    (f"conv_1",conv_layer(nh, nh, 3, stride=1)), # stride 1 !!!
#                    (f"conv_2",conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False)) ### act!!!
#         ]
#         if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
#         self.convs = nn.Sequential(OrderedDict(layers))
#         self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
#         self.merge =act_fn

#     def forward(self, x): 
#         o = self.reduce(x)
#         return self.merge(self.convs(o) + self.idconv(o))

In [ ]:
# export
# Still no name - just New block YET!
class NewResBlock(nn.Module):
    def __init__(self, expansion, ni, nh, stride=1, 
                 conv_layer=ConvLayer, act_fn=act_fn,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False,sym=False, zero_bn=True):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        self.reduce = noop if stride==1 else pool
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, stride=1)), # stride 1 !!!
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False))
        ] if expansion == 1 else [
                   (f"conv_0",conv_layer(ni, nh, 1)),
                   (f"conv_1",conv_layer(nh, nh, 3, stride=1)), # stride 1 !!!
                   (f"conv_2",conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False)) ### act!!!
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.merge =act_fn

    def forward(self, x): 
        o = self.reduce(x)
        return self.merge(self.convs(o) + self.idconv(o))

In [ ]:
# hide
bl = NewResBlock(1,64,64,sa=True)
bl

NewResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (merge): ReLU(inplace=True)
)

In [ ]:
# hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
# hide
bl = NewResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU())
bl

NewResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Leaky

In [ ]:
# hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


# class Consructor

In [ ]:
class Constructor():
    def __init__(self, **kwargs): 
        self._name = type(self).__name__
        self.__dict__.update(locals())
        self.__dict__.update(self.__dict__.pop('kwargs'))
        
    def __repr__(self): return f"{self._name}"  
    
    def register(self, model):
            self._model = model
            for attr in self.__dict__.copy():
                if not attr.startswith('_') and not attr=='self':
                    if not hasattr(model, attr):
                        print(f"setting {attr}")
                        setattr(model, attr, self.__dict__[attr])
                    self.__dict__.pop(attr)


In [ ]:
cons = Constructor(x=1)

In [ ]:
cons

Constructor

In [ ]:
cons.__dict__

{'_name': 'Constructor', 'self': Constructor, 'x': 1}

## tsts

In [ ]:
cons_2 = Constructor(bn_1st=True)

In [ ]:
cons

constr_base

In [ ]:
cons._name

'constr_base'

In [ ]:
cons.__dict__

{'bn_1st': False, '_name': 'constr_base'}

In [ ]:
cons.aaa

'base not found aaa'

In [ ]:
cons

In [ ]:
class _ConvL(Constructor):
#     def __init__(self, parent=cons, conv_layer=ConvLayer, 
    def __init__(self, parent, conv_layer=ConvLayer, 
                 act_fn=nn.ReLU(inplace=True), bn_1st=True):
        super().__init__()
#         in_args = locals()
#         print(in_args)
#         del in_args['self']
#         del in_args['parent']
#         del in_args['__class__']
#         print(in_args)
#         self.parent = parent
        
#         for arg in in_args:
#             if not hasattr(self.parent, arg):
#                 setattr(self.parent, arg, in_args[arg])
        
#     def __getattr__(self, k): return getattr(self.parent, k)
# #         act_fn=act_fn, bn_1st=True, 
# #             padding=None, bias=False, groups=1
# #         pass

#     def __call__(self, ni, nf, ks=3, stride=1, 
#             act=True,  bn_layer=True, zero_bn=False):
#         return self.conv_layer(ni, nf, ks=3, stride=1, 
# #         ConvLayer(ni, nf, ks=3, stride=1, 
# #         NewConvLayer(ni, nf, ks=3, stride=1, 
#             act=True,  bn_layer=True, zero_bn=False)
        

In [ ]:
l = _ConvL(cons)

In [ ]:
l.__getattr__

<bound method Constructor.__getattr__ of _ConvL>

In [ ]:
l.__dict__

{'self': _ConvL, '_name': '_ConvL'}

In [ ]:
l.parent = cons

In [ ]:
cons

In [ ]:
l.parent.__dict__

{'bn_1st': False, '_name': 'constr_base'}

In [ ]:
l.parent

In [ ]:
l.bn_1st

True

In [ ]:
l.aaa

'base not found aaa'

In [ ]:
l.parent

In [ ]:
getattr(cons, 'bn_1st')

False

In [ ]:
cons.bn_1st

False

In [ ]:
getattr(l.parent, 'bn_1st')

True

In [ ]:
getattr(l, 'bn_1st')

True

In [ ]:
l.__getattr__

<bound method _ConvL.__getattr__ of <__main__._ConvL object at 0x7f2a1b702c10>>

In [ ]:
l.__getattr__ = lambda _, k: getattr(cons_2, k)

In [ ]:
l.bn_1st

False

In [ ]:
__getattribute__()

In [ ]:
l.aaa

'base not found aaa'

In [ ]:
cons_2.bn_1st

True

In [ ]:
getattr(cons_2, 'aaa')

'qqq'

In [ ]:
cons_2.aaa = 'qqq'

In [ ]:
l.__getattr__

<function __main__.<lambda>(_, k)>

In [ ]:
l(64,128)

ConvLayer(
  (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
ConvLayer(64,128)

ConvLayer(
  (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
cons.__dict__

{'bn_1st': False, '_name': 'constr_base'}

In [ ]:
hasattr(cons, 'act_fn')

False

In [ ]:
cons.act_fn

LeakyReLU(negative_slope=0.01)

# NewConvLayer

In [ ]:
# # export
# class NewConvLayer(Constructor):
#     """Basic conv layers block"""
#     def __init__(self, model, act_fn=act_fn, bn_1st=True, 
#             padding=None, bias=False, groups=1, norm=nn.BatchNorm2d, **kwargs):
# #         self.__dict__ = locals()
#         in_args = locals()
# #         print(in_args)
#         del in_args['self']
#         del in_args['model']
# #         del in_args['__class__']
# #         print(in_args)
#         self._model = model
#         self._name = 'NewConvLayer'
#         for arg in in_args:
# #             print(arg)
#             if not hasattr(self._model, arg):
#                 setattr(self._model, arg, in_args[arg])
# #             else: print('exist')
        
#     def __getattr__(self, k): return getattr(self._model, k)
        
#     def __call__(self, ni, nf, ks=3, stride=1, 
#             act=True,  bn_layer=True, zero_bn=False, **kwargs): # todo check kwargs
#         padding = ks//2 if self.padding==None else self.padding
#         layers = [('conv', nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, 
#                                      bias=self.bias, groups=self.groups))]
#         act_bn = [('act_fn', self.act_fn)] if act else []
#         if bn_layer:
#             bn = self.norm(nf)
#             nn.init.constant_(bn.weight, 0. if zero_bn else 1.) 
#             act_bn += [('bn', bn)]
#         if self.bn_1st: act_bn.reverse()
#         layers += act_bn   
#         return nn.Sequential(OrderedDict(layers))

In [ ]:
#     def register(self, model):
#         self._model = model
#         for attr in self.__dict__.copy():
#             if not attr.startswith('_') and not attr=='self':
#                 if not hasattr(model, attr):           
#                     setattr(model, attr, self.__dict__[attr])
#                 self.__dict__.pop(attr)
    
#     def __getattr__(self, k): 
#         if hasattr(self, '_model'): return getattr(self._model, k)
    

In [ ]:
# export
class NewConvLayer(Constructor):
    """Basic conv layers block"""
    def __init__(self, bn_1st=True, act_fn=act_fn, norm=nn.BatchNorm2d,  
            padding=None, bias=False, groups=1,  **kwargs):
        super().__init__()
        self.__dict__.update(locals())
        self.__dict__.update(self.__dict__.pop('kwargs'))
    
    def __call__(self, ni, nf, ks=3, stride=1, 
            act=True,  bn_layer=True, zero_bn=False, **kwargs): # todo check kwargs
        padding = ks//2 if self.padding==None else self.padding
        layers = [('conv', nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, 
                                     bias=self.bias, groups=self.groups))]
        act_bn = [('act_fn', self.act_fn)] if act else []
        if bn_layer:
            bn = self.norm(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.) 
            act_bn += [('bn', bn)]
        if self.bn_1st: act_bn.reverse()
        layers += act_bn   
        return nn.Sequential(OrderedDict(layers))
    
    def __getattr__(self, k): 
        if hasattr(self, '_model'):
            return getattr(self._model, k)

In [ ]:
# _conv_layer = NewConvLayer(cons)
_conv_layer = NewConvLayer(z=100)

In [ ]:
_conv_layer

NewConvLayer

In [ ]:
_conv_layer.__dict__

{'_name': 'NewConvLayer',
 'self': NewConvLayer,
 'act_fn': ReLU(inplace=True),
 'bn_1st': True,
 'padding': None,
 'bias': False,
 'groups': 1,
 'norm': torch.nn.modules.batchnorm.BatchNorm2d,
 '__class__': __main__.NewConvLayer,
 'z': 100}

In [ ]:
cons = Constructor(bn_1st=False)

In [ ]:
cons.bn_1st

False

In [ ]:
_conv_layer.register(cons)

setting act_fn
setting padding
setting bias
setting groups
setting norm
setting z


In [ ]:
_conv_layer._model.bn_1st

False

In [ ]:
_conv_layer.bn_1st

False

In [ ]:
_conv_layer._model.act_fn

ReLU(inplace=True)

In [ ]:
_conv_layer

NewConvLayer

In [ ]:
cons.__dict__

{'_name': 'Constructor',
 'self': Constructor,
 'bn_1st': False,
 'act_fn': ReLU(inplace=True),
 'padding': None,
 'bias': False,
 'groups': 1,
 'norm': torch.nn.modules.batchnorm.BatchNorm2d,
 'z': 100}

In [ ]:
_conv_layer.__dict__

{'_name': 'NewConvLayer',
 'self': NewConvLayer,
 '__class__': __main__.NewConvLayer,
 '_model': Constructor}

In [ ]:
_conv_layer._model

Constructor

In [ ]:
_conv_layer.act_fn

ReLU(inplace=True)

In [ ]:
_conv_layer.bn_1st

False

In [ ]:
_conv_layer._model.__dict__

{'_name': 'Constructor',
 'self': Constructor,
 'bn_1st': False,
 'act_fn': ReLU(inplace=True),
 'padding': None,
 'bias': False,
 'groups': 1,
 'norm': torch.nn.modules.batchnorm.BatchNorm2d,
 'z': 100}

In [ ]:
_conv_layer(64,128,1,2,)

Sequential(
  (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (act_fn): ReLU(inplace=True)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
cons.act_fn = nn.SELU()

In [ ]:
_conv_layer(64,128,1,2,)

Sequential(
  (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (act_fn): SELU()
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
cons.__dict__

{'_name': 'Constructor',
 'self': Constructor,
 'bn_1st': False,
 'act_fn': SELU(),
 'padding': None,
 'bias': False,
 'groups': 1,
 'norm': torch.nn.modules.batchnorm.BatchNorm2d,
 'z': 100}

In [ ]:
_conv_layer.__dict__

{'_name': 'NewConvLayer',
 'self': NewConvLayer,
 '__class__': __main__.NewConvLayer,
 '_model': Constructor}

In [ ]:
cons.act_fn = nn.LeakyReLU()

In [ ]:
_conv_layer(64,128,1,2,)

Sequential(
  (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (act_fn): LeakyReLU(negative_slope=0.01)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
cons.bn_1st = True

In [ ]:
_conv_layer(64,128,1,2,)

Sequential(
  (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [ ]:
# v8
class Net():
    def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000, name='Net'):
        super().__init__()
        self.name = name
        self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers # todo setter for expansion
        self.stem_sizes = [c_in,32,32,64]
        self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.stem_bn_end = False
        self.norm = nn.BatchNorm2d
        self.act_fn=nn.ReLU(inplace=True)
        self.pool = nn.AvgPool2d(2, ceil_mode=True)
        self.sa=False
        self.bn_1st = True
        self.zero_bn=True
        self._init_cnn = init_cnn
        self.block = ResBlock
        self.conv_layer = ConvLayer
        
    @property
    def block_szs(self):
        return [64//self.expansion,64,128,256,512] +[256]*(len(self.layers)-4) 

    @property
    def stem(self):
        return self._make_stem()
    @property
    def head(self):
        return self._make_head()
    @property
    def body(self):
        return self._make_body()
    
    def _make_stem(self):
        stem = [(f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], 
                    stride=2 if i==0 else 1, 
                    norm=(not self.stem_bn_end) if i==(len(self.stem_sizes)-2) else True,
                    act_fn=self.act_fn, bn_1st=self.bn_1st))
                for i in range(len(self.stem_sizes)-1)]
        stem.append(('stem_pool', self.stem_pool))
        if self.stem_bn_end: stem.append(('norm', self.norm(self.stem_sizes[-1])))
        return nn.Sequential(OrderedDict(stem))
    
    def _make_head(self):
        head = [('pool', nn.AdaptiveAvgPool2d(1)),
                ('flat', Flatten()),
                ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
        return nn.Sequential(OrderedDict(head))
    
    def _make_body(self):
        blocks = [(f"l_{i}", self._make_layer(self.block_szs[i], self.block_szs[i+1], l, 
                        1 if i==0 else 2, self.sa if i==0 else False))
                  for i,l in enumerate(self.layers)]
        return nn.Sequential(OrderedDict(blocks))
    
    def _make_layer(self,ni,nf,blocks,stride,sa):
        return nn.Sequential(OrderedDict(
            [(f"bl_{i}", self.block(self.expansion, ni if i==0 else nf, nf, 
                    stride if i==0 else 1, sa=sa if i==blocks-1 else False,
                    conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool,zero_bn=self.zero_bn))
              for i in range(blocks)]))
    
    def __call__(self):
        model = nn.Sequential(OrderedDict([
            ('stem', self.stem),
            ('body', self.body),
            ('head', self.head)
        ]))
        self._init_cnn(model)
        model.extra_repr = lambda : f"model {self.name}"
        return model
    def __repr__(self):
        return f" constr {self.name}"

# Net class.

In [ ]:
# export
# v9
class Net():
    def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000, name='Net'):
        super().__init__()
        self.name = name
        self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers # todo setter for expansion
        self.stem_sizes = [c_in,32,32,64]
        self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.stem_bn_end = False
        self.norm = nn.BatchNorm2d
        self.act_fn=nn.ReLU(inplace=True)
        self.pool = nn.AvgPool2d(2, ceil_mode=True)
        self.sa=False
        self.bn_1st = True
        self.zero_bn=True
        self._init_cnn = init_cnn
        self.block = ResBlock
        
#         self.conv_layer = ConvLayer
        self.conv_layer = NewConvLayer
    
    @property
    def conv_layer(self): return self._conv_layer
    @conv_layer.setter
    def conv_layer(self, f):
        self._conv_layer = f()
        self._conv_layer.register(self)
        
    
    @property
    def block_szs(self):
        return [64//self.expansion,64,128,256,512] +[256]*(len(self.layers)-4) 

    @property
    def stem(self):
        return self._make_stem()
    @property
    def head(self):
        return self._make_head()
    @property
    def body(self):
        return self._make_body()
 
    def _block(self, ni, nh, stride=1, sa=False,sym=False):
        return self.block(self.expansion, ni, nh, stride, 
                 self.conv_layer, self.act_fn,
                 pool=self.pool, sa=sa,sym=sym, zero_bn=self.zero_bn)
    
    
    def _make_stem(self):
        stem = [(f"conv_{i}", self._conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], 
                    stride=2 if i==0 else 1, 
                    bn_layer=(not self.stem_bn_end) if i==(len(self.stem_sizes)-2) else True,))
                for i in range(len(self.stem_sizes)-1)]
        stem.append(('stem_pool', self.stem_pool))
        if self.stem_bn_end: stem.append(('norm', self.norm(self.stem_sizes[-1])))
        return nn.Sequential(OrderedDict(stem))
    
    def _make_head(self):
        head = [('pool', nn.AdaptiveAvgPool2d(1)),
                ('flat', Flatten()),
                ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
        return nn.Sequential(OrderedDict(head))
    
    def _make_body(self):
        blocks = [(f"l_{i}", self._make_layer(self.block_szs[i], self.block_szs[i+1], l, 
                        1 if i==0 else 2, self.sa if i==0 else False))
                  for i,l in enumerate(self.layers)]
        return nn.Sequential(OrderedDict(blocks))
    
    def _make_layer(self,ni,nf,blocks,stride,sa):
        return nn.Sequential(OrderedDict(
            [(f"bl_{i}", self._block(ni if i==0 else nf, nf, 
                    stride if i==0 else 1, sa=sa if i==blocks-1 else False))
              for i in range(blocks)]))
    
    def __call__(self):
        model = nn.Sequential(OrderedDict([
            ('stem', self.stem),
            ('body', self.body),
            ('head', self.head)
        ]))
        self._init_cnn(model)
        model.extra_repr = lambda : f"model {self.name}"
        return model
    def __repr__(self):
        return f" constr {self.name}"

In [ ]:
model  = Net()

setting padding
setting bias
setting groups


In [ ]:
model.conv_layer.act_fn

ReLU(inplace=True)

In [ ]:
model.conv_layer

NewConvLayer

In [ ]:
# model.conv_layer = NewConvLayer

In [ ]:
model.conv_layer(64,128)

Sequential(
  (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
model

 constr Net

In [ ]:
model.block_szs

[64, 64, 128, 256, 512]

In [ ]:
model.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
# hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = model.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
model.bn_1st = False

In [ ]:
model.act_fn =nn.LeakyReLU(inplace=True)

In [ ]:
model.conv_layer.act_fn

LeakyReLU(negative_slope=0.01, inplace=True)

In [ ]:
model.sa = True

In [ ]:
# model.conv_layer =NewConvLayer

In [ ]:
model.conv_layer.bn_1st

False

In [ ]:
model.conv_layer.act_fn

LeakyReLU(negative_slope=0.01, inplace=True)

In [ ]:
model.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
# model.block = NewResBlock
model.block = ResBlock

In [ ]:
model.bn_1st

False

In [ ]:
model.block(1,64,128,2,)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (idconv): ConvLayer(
    (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
model.body.l_1

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): Sequential(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): Sequential(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): Sequential(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (bl_1): ResBlock(
    (convs): Sequential(
      (conv_0): Sequential(
        (conv): Conv2d(12

In [ ]:
model._block(64,64)

ResBlock(
  (convs): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
)

In [ ]:
model.block(1,64,64,2)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
# hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
model.block = NewResBlock

In [ ]:
# hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_1.bl_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 128, 16, 16]), f"size"

torch.Size([16, 128, 16, 16])


In [ ]:
# hide
model.body.l_1.bl_0

NewResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): Sequential(
    (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): LeakyReLU(negative_slope=0.01, inplace=True)
)

In [ ]:
# model = Net(expansion=4)
model.expansion = 4

In [ ]:
model.stem_bn_end = True

In [ ]:
model.bn_1st = True

In [ ]:
model.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
model.pool

AvgPool2d(kernel_size=2, stride=2, padding=0)

In [ ]:
model.pool = nn.MaxPool2d(3)

In [ ]:
model.body.l_1.bl_0

NewResBlock(
  (reduce): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (convs): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (conv_2): Sequential(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): Sequential(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512,

In [ ]:
model.block = NewResBlock

In [ ]:
# hide
model.body.l_1.bl_0

NewResBlock(
  (reduce): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (convs): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (conv_2): Sequential(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): Sequential(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512,

In [ ]:
# hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = model.body.l_1.bl_0(xb)
print(y.shape)
# assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 10, 10])


In [ ]:
m = model()

In [ ]:
# hide
m

Sequential(
  model Net
  (stem): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (conv_2): Sequential(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (l

In [ ]:
# hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = m(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])


In [ ]:
model.stem_sizes = [3, 32, 32, 64, 64]

In [ ]:
# hide
model.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (conv_3): Sequential(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (stem_poo

In [ ]:
# hide
model.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten()
  (fc): Linear(in_features=2048, out_features=1000, bias=True)
)

## xresnet constructor

In [ ]:
# export
me = sys.modules[__name__]
for n,e,l in [[ 18 , 1, [2,2,2 ,2] ],
    [ 34 , 1, [3,4,6 ,3] ],
    [ 50 , 4, [3,4,6 ,3] ],
    [ 101, 4, [3,4,23,3] ],
    [ 152, 4, [3,8,36,3] ],]:
    name = f'net{n}'
    setattr(me, name, partial(Net, expansion=e, layers=l, name=name))
xresnet50      = partial(Net, expansion=4, layers=[3, 4,  6, 3], name='xresnet50')

In [ ]:
m = xresnet50(c_out=10)

In [ ]:
m, m.c_out

( constr xresnet50, 10)

# end
model_constructor
by ayasyrev

In [ ]:
# hide
from nbdev.export import *
notebook2script()